### Data Preprocessing Notebook for Initial Livestock Data
- Author: Juan Antonio Robledo Lara
- Objective: This notebooks allows the user to test the data preprocessing script for initial livestock data so its easier to create new updates if necessary.
- Notes: Refer to the main script `initial_lvst_preprocessing.py` to run a new update of the livestock data.

In [65]:
import pandas as pd
import os
import numpy as np

In [66]:
SCRIPT_DIR_PATH = os.getcwd()
PROJECT_DIR_PATH = os.path.dirname(SCRIPT_DIR_PATH)
AFOLU_DIR_PATH = os.path.join(PROJECT_DIR_PATH, 'AFOLU')
LVST_RAW_DATA_DIR_PATH = os.path.join(AFOLU_DIR_PATH, 'pop_lvst_raw_data')

In [67]:
lvst_raw_df = pd.read_csv(os.path.join(LVST_RAW_DATA_DIR_PATH, 'FAOSTAT_livestock_data.csv'))
lvst_raw_df.head()

,Domain Code,Domain,Area Code (ISO3),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,AFG,Afghanistan,5111,Stocks,2132.0,Asses,1961,1961,An,1300000.0,A,Official figure,NaN
1,QCL,Crops and livestock products,AFG,Afghanistan,5111,Stocks,2132.0,Asses,1962,1962,An,851850.0,A,Official figure,NaN
2,QCL,Crops and livestock products,AFG,Afghanistan,5111,Stocks,2132.0,Asses,1963,1963,An,1001112.0,A,Official figure,NaN
3,QCL,Crops and livestock products,AFG,Afghanistan,5111,Stocks,2132.0,Asses,1964,1964,An,1150000.0,E,Estimated value,NaN
4,QCL,Crops and livestock products,AFG,Afghanistan,5111,Stocks,2132.0,Asses,1965,1965,An,1300000.0,A,Official figure,NaN


In [68]:
# Drop irrelevant columns
lvst_df = lvst_raw_df[['Area Code (ISO3)', 'Area', 'Year', 'Item', 'Value', 'Unit']]
lvst_df = lvst_df.rename(columns={'Area Code (ISO3)':'iso_code3', 'Area':'Nation'})
lvst_df.head()

,iso_code3,Nation,Year,Item,Value,Unit
0,AFG,Afghanistan,1961,Asses,1300000.0,An
1,AFG,Afghanistan,1962,Asses,851850.0,An
2,AFG,Afghanistan,1963,Asses,1001112.0,An
3,AFG,Afghanistan,1964,Asses,1150000.0,An
4,AFG,Afghanistan,1965,Asses,1300000.0,An


In [69]:
# Checking if there are missing iso_code3 values
print(lvst_df.info())
print(lvst_df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108897 entries, 0 to 108896
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   iso_code3  108897 non-null  object 
 1   Nation     108897 non-null  object 
 2   Year       108897 non-null  int64  
 3   Item       108897 non-null  object 
 4   Value      105939 non-null  float64
 5   Unit       108897 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 5.0+ MB
None
iso_code3       0
Nation          0
Year            0
Item            0
Value        2958
Unit            0
dtype: int64


In [70]:
# Checking that the df looks good using IRAN pigs as a example which should be all 0.
lvst_df[(lvst_df.iso_code3 == 'IRN') & (lvst_df.Item == 'Swine / pigs')]

,iso_code3,Nation,Year,Item,Value,Unit
48778,IRN,Iran (Islamic Republic of),1961,Swine / pigs,55000.0,An
48779,IRN,Iran (Islamic Republic of),1962,Swine / pigs,58000.0,An
48780,IRN,Iran (Islamic Republic of),1963,Swine / pigs,52000.0,An
48781,IRN,Iran (Islamic Republic of),1964,Swine / pigs,50000.0,An
48782,IRN,Iran (Islamic Republic of),1965,Swine / pigs,50000.0,An
...,...,...,...,...,...,...
48836,IRN,Iran (Islamic Republic of),2019,Swine / pigs,NaN,An
48837,IRN,Iran (Islamic Republic of),2020,Swine / pigs,NaN,An
48838,IRN,Iran (Islamic Republic of),2021,Swine / pigs,NaN,An
48839,IRN,Iran (Islamic Republic of),2022,Swine / pigs,NaN,An


In [71]:
# Read the FAO mapping csv
FAO_mapping_df = pd.read_csv(os.path.join(LVST_RAW_DATA_DIR_PATH, 'fao_product_demand_categories_for_import_export.csv'))
FAO_mapping_df.head()

,fao_category,sisepuede_demand_subsector,sisepuede_demand_category,agg_to_fao_category
0,Chocolate products nes,agrc,bevs_and_spices,NaN
1,Cinnamon (cannella),agrc,bevs_and_spices,NaN
2,Cloves,agrc,bevs_and_spices,NaN
3,"Cocoa, beans",agrc,bevs_and_spices,NaN
4,"Cocoa, butter",agrc,bevs_and_spices,NaN


In [72]:
# Filter the data to only get the livestock items in the cw file
FAO_mapping_df = FAO_mapping_df[FAO_mapping_df['sisepuede_demand_subsector'].str.contains('lvst')].copy()
FAO_mapping_df = FAO_mapping_df.reset_index(drop=True)
FAO_mapping_df = FAO_mapping_df.drop(columns=['sisepuede_demand_subsector', 'agg_to_fao_category'])
FAO_mapping_df

,fao_category,sisepuede_demand_category
0,Buffalo,buffalo
1,"Fat, buffaloes",buffalo
2,"Hides, buffalo, fresh",buffalo
3,"Hides, buffalo, wet salted",buffalo
4,"Meat, buffalo",buffalo
...,...,...
110,"Skins, sheep, wet salted",sheep
111,"Skins, sheep, with wool",sheep
112,"Wool, degreased",sheep
113,"Wool, greasy",sheep


In [73]:
lvst_df.Item.unique()

array(['Asses', 'Camels', 'Cattle', 'Chickens', 'Goats', 'Horses',
       'Mules and hinnies', 'Sheep', 'Bees', 'Buffalo', 'Ducks', 'Geese',
       'Swine / pigs', 'Turkeys', 'Rabbits and hares', 'Other camelids',
       'Other rodents', 'Other birds'], dtype=object)

In [74]:
# merge lvst_df with FAO_mapping_df
lvst_df = lvst_df.merge(FAO_mapping_df, how='inner', left_on='Item', right_on='fao_category')
# Drop the item column
lvst_df = lvst_df.drop(columns=['Item'])
lvst_df.head()

,iso_code3,Nation,Year,Value,Unit,fao_category,sisepuede_demand_category
0,AFG,Afghanistan,1961,1300000.0,An,Asses,mules
1,AFG,Afghanistan,1962,851850.0,An,Asses,mules
2,AFG,Afghanistan,1963,1001112.0,An,Asses,mules
3,AFG,Afghanistan,1964,1150000.0,An,Asses,mules
4,AFG,Afghanistan,1965,1300000.0,An,Asses,mules


In [75]:
lvst_df[['fao_category', 'Unit']].drop_duplicates()

,fao_category,Unit
0,Asses,An
63,Camels,An
126,Cattle,An
189,Chickens,1000 An
252,Goats,An
315,Horses,An
378,Mules and hinnies,An
441,Sheep,An
567,Buffalo,An
756,Ducks,1000 An


In [76]:
lvst_df[lvst_df['fao_category'].isin(['Chickens', 'Ducks', 'Turkeys'])].head()

,iso_code3,Nation,Year,Value,Unit,fao_category,sisepuede_demand_category
189,AFG,Afghanistan,1961,4700.0,1000 An,Chickens,chickens
190,AFG,Afghanistan,1962,4900.0,1000 An,Chickens,chickens
191,AFG,Afghanistan,1963,5000.0,1000 An,Chickens,chickens
192,AFG,Afghanistan,1964,5300.0,1000 An,Chickens,chickens
193,AFG,Afghanistan,1965,5500.0,1000 An,Chickens,chickens


In [77]:
# Multiply by a factor of 1000 Chickens, Ducks and Turkeys and drop Unit col
lvst_df.loc[lvst_df['fao_category'].isin(['Chickens', 'Ducks', 'Turkeys']), 'Value'] *= 1000
lvst_df = lvst_df.drop(columns=['Unit'])
lvst_df.head()

,iso_code3,Nation,Year,Value,fao_category,sisepuede_demand_category
0,AFG,Afghanistan,1961,1300000.0,Asses,mules
1,AFG,Afghanistan,1962,851850.0,Asses,mules
2,AFG,Afghanistan,1963,1001112.0,Asses,mules
3,AFG,Afghanistan,1964,1150000.0,Asses,mules
4,AFG,Afghanistan,1965,1300000.0,Asses,mules


In [78]:
lvst_df[lvst_df['fao_category'].isin(['Chickens', 'Ducks', 'Turkeys'])].head()

,iso_code3,Nation,Year,Value,fao_category,sisepuede_demand_category
189,AFG,Afghanistan,1961,4700000.0,Chickens,chickens
190,AFG,Afghanistan,1962,4900000.0,Chickens,chickens
191,AFG,Afghanistan,1963,5000000.0,Chickens,chickens
192,AFG,Afghanistan,1964,5300000.0,Chickens,chickens
193,AFG,Afghanistan,1965,5500000.0,Chickens,chickens


In [79]:
lvst_df[(lvst_df['Year'] == 2000) & (lvst_df['iso_code3'] == 'AFG') & (lvst_df['sisepuede_demand_category'] == 'horses')]

,iso_code3,Nation,Year,Value,fao_category,sisepuede_demand_category
102,AFG,Afghanistan,2000,224000.0,Camels,horses
354,AFG,Afghanistan,2000,162000.0,Horses,horses


In [80]:
# assume your DataFrame is called df
df_grouped = (
    lvst_df
    .groupby(
        ['iso_code3', 'Year', 'sisepuede_demand_category'],
        as_index=False  # keep the grouping columns as columns, not index
    )[['Value']]      # select the column(s) you want to aggregate
    .sum()
)

df_grouped.head()


,iso_code3,Year,sisepuede_demand_category,Value
0,AFG,1961,cattle_nondairy,2900000.0
1,AFG,1961,chickens,4700000.0
2,AFG,1961,goats,4200000.0
3,AFG,1961,horses,526841.0
4,AFG,1961,mules,1320000.0


In [81]:
df_grouped[(df_grouped['Year'] == 2000) & (df_grouped['iso_code3'] == 'AFG') & (df_grouped['sisepuede_demand_category'] == 'horses')]

,iso_code3,Year,sisepuede_demand_category,Value
237,AFG,2000,horses,386000.0


In [82]:
iso_codes = df_grouped.iso_code3.unique()
demand_cat = df_grouped.sisepuede_demand_category.unique()


#array of years from 2011 to 2022
years = np.arange(2011, 2023)

combinations_df = pd.DataFrame(
    [(iso, year, cat) for iso in iso_codes for year in years for cat in demand_cat],
    columns=['iso_code3', 'Year', 'sisepuede_demand_category']
)

combinations_df.head()

,iso_code3,Year,sisepuede_demand_category
0,AFG,2011,cattle_nondairy
1,AFG,2011,chickens
2,AFG,2011,goats
3,AFG,2011,horses
4,AFG,2011,mules


In [83]:
# Get unique iso_code3 and Nation pairs
unique_iso_nation_pairs = lvst_df[['iso_code3', 'Nation']].drop_duplicates()
unique_iso_nation_pairs

,iso_code3,Nation
0,AFG,Afghanistan
504,ALB,Albania
1197,DZA,Algeria
1890,AGO,Angola
2331,ATG,Antigua and Barbuda
...,...,...
91823,VNM,Viet Nam
92264,YEM,Yemen
92705,F248,Yugoslav SFR
93015,ZMB,Zambia


In [84]:
template_df = pd.merge(combinations_df, df_grouped, on=['iso_code3', 'Year', 'sisepuede_demand_category'], how='left')
template_df = pd.merge(template_df, unique_iso_nation_pairs, on='iso_code3', how='left')
template_df

,iso_code3,Year,sisepuede_demand_category,Value,Nation
0,AFG,2011,cattle_nondairy,5524000.0,Afghanistan
1,AFG,2011,chickens,13378000.0,Afghanistan
2,AFG,2011,goats,7635000.0,Afghanistan
3,AFG,2011,horses,353000.0,Afghanistan
4,AFG,2011,mules,1491000.0,Afghanistan
...,...,...,...,...,...
20059,ZWE,2022,horses,28171.0,Zimbabwe
20060,ZWE,2022,mules,645290.0,Zimbabwe
20061,ZWE,2022,sheep,431957.0,Zimbabwe
20062,ZWE,2022,pigs,362732.0,Zimbabwe


In [85]:
template_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20064 entries, 0 to 20063
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   iso_code3                  20064 non-null  object 
 1   Year                       20064 non-null  int64  
 2   sisepuede_demand_category  20064 non-null  object 
 3   Value                      15207 non-null  float64
 4   Nation                     20064 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 783.9+ KB


In [86]:
template_df['Value_ffill'] = template_df.groupby(
    ['iso_code3','sisepuede_demand_category']
)['Value'].transform('ffill')

template_df['Value_imputed'] = template_df.groupby(
    ['iso_code3','sisepuede_demand_category']
)['Value_ffill'].transform('bfill')

template_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20064 entries, 0 to 20063
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   iso_code3                  20064 non-null  object 
 1   Year                       20064 non-null  int64  
 2   sisepuede_demand_category  20064 non-null  object 
 3   Value                      15207 non-null  float64
 4   Nation                     20064 non-null  object 
 5   Value_ffill                15440 non-null  float64
 6   Value_imputed              15624 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.1+ MB


In [87]:
# Drop records with NaNs in Value imputed
template_df = template_df.dropna(subset='Value_imputed')

# Drop other value cols
template_df = template_df.drop(columns=['Value', 'Value_ffill'])

template_df = template_df.reset_index(drop=True)

template_df.head()

,iso_code3,Year,sisepuede_demand_category,Nation,Value_imputed
0,AFG,2011,cattle_nondairy,Afghanistan,5524000.0
1,AFG,2011,chickens,Afghanistan,13378000.0
2,AFG,2011,goats,Afghanistan,7635000.0
3,AFG,2011,horses,Afghanistan,353000.0
4,AFG,2011,mules,Afghanistan,1491000.0


In [88]:
template_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15624 entries, 0 to 15623
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   iso_code3                  15624 non-null  object 
 1   Year                       15624 non-null  int64  
 2   sisepuede_demand_category  15624 non-null  object 
 3   Nation                     15624 non-null  object 
 4   Value_imputed              15624 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 610.4+ KB


In [89]:
# Pivot
df_wide = (
    template_df
    .pivot_table(
        index=['iso_code3', 'Year', 'Nation'],           # the keys to remain as rows
        columns='sisepuede_demand_category',   # the values that become new columns
        values='Value_imputed',                        # the cell values
        aggfunc='sum',                         # how to aggregate duplicates
        fill_value=0                           # replace missing with 0 (if you like)
    )
    .reset_index()                             # bring iso_code3 and Year back as columns
)

# optional: flatten the column index
df_wide.columns.name = None

df_wide

,iso_code3,Year,Nation,buffalo,cattle_nondairy,chickens,goats,horses,mules,pigs,sheep
0,AFG,2011,Afghanistan,0.0,5524000.0,13378000.0,7635000.0,353000.0,1491000.0,0.0,14262000.0
1,AFG,2012,Afghanistan,0.0,5244000.0,13212000.0,7311000.0,352000.0,1447000.0,0.0,13820000.0
2,AFG,2013,Afghanistan,0.0,5235000.0,12053000.0,7037000.0,341000.0,1472000.0,0.0,13141000.0
3,AFG,2014,Afghanistan,0.0,5349000.0,11098000.0,7059000.0,342000.0,1465000.0,0.0,13485000.0
4,AFG,2015,Afghanistan,0.0,5261000.0,11863000.0,7723000.0,343000.0,1505500.0,0.0,13218000.0
...,...,...,...,...,...,...,...,...,...,...,...
2383,ZWE,2018,Zimbabwe,0.0,5320047.0,15008000.0,5054753.0,28151.0,539534.0,231459.0,355906.0
2384,ZWE,2019,Zimbabwe,0.0,5057563.0,17116000.0,5721322.0,28133.0,520751.0,182896.0,352608.0
2385,ZWE,2020,Zimbabwe,0.0,5195154.0,16318000.0,5772766.0,28242.0,562855.0,269508.0,376033.0
2386,ZWE,2021,Zimbabwe,0.0,5478648.0,18113000.0,5928807.0,28163.0,599316.0,278106.0,361516.0


In [90]:
# rename cols by adding a pop_lvst_initial prefix
df_wide.columns = ['_'.join(['pop_lvst_initial', str(col)]) if col not in ['iso_code3', 'Year', 'Nation'] else col for col in df_wide.columns]
df_wide.head()

,iso_code3,Year,Nation,pop_lvst_initial_buffalo,pop_lvst_initial_cattle_nondairy,pop_lvst_initial_chickens,pop_lvst_initial_goats,pop_lvst_initial_horses,pop_lvst_initial_mules,pop_lvst_initial_pigs,pop_lvst_initial_sheep
0,AFG,2011,Afghanistan,0.0,5524000.0,13378000.0,7635000.0,353000.0,1491000.0,0.0,14262000.0
1,AFG,2012,Afghanistan,0.0,5244000.0,13212000.0,7311000.0,352000.0,1447000.0,0.0,13820000.0
2,AFG,2013,Afghanistan,0.0,5235000.0,12053000.0,7037000.0,341000.0,1472000.0,0.0,13141000.0
3,AFG,2014,Afghanistan,0.0,5349000.0,11098000.0,7059000.0,342000.0,1465000.0,0.0,13485000.0
4,AFG,2015,Afghanistan,0.0,5261000.0,11863000.0,7723000.0,343000.0,1505500.0,0.0,13218000.0


In [91]:
# add cattle_dairy column
df_wide['pop_lvst_initial_cattle_dairy'] = df_wide['pop_lvst_initial_cattle_nondairy'].copy()
df_wide.head() 

,iso_code3,Year,Nation,pop_lvst_initial_buffalo,pop_lvst_initial_cattle_nondairy,pop_lvst_initial_chickens,pop_lvst_initial_goats,pop_lvst_initial_horses,pop_lvst_initial_mules,pop_lvst_initial_pigs,pop_lvst_initial_sheep,pop_lvst_initial_cattle_dairy
0,AFG,2011,Afghanistan,0.0,5524000.0,13378000.0,7635000.0,353000.0,1491000.0,0.0,14262000.0,5524000.0
1,AFG,2012,Afghanistan,0.0,5244000.0,13212000.0,7311000.0,352000.0,1447000.0,0.0,13820000.0,5244000.0
2,AFG,2013,Afghanistan,0.0,5235000.0,12053000.0,7037000.0,341000.0,1472000.0,0.0,13141000.0,5235000.0
3,AFG,2014,Afghanistan,0.0,5349000.0,11098000.0,7059000.0,342000.0,1465000.0,0.0,13485000.0,5349000.0
4,AFG,2015,Afghanistan,0.0,5261000.0,11863000.0,7723000.0,343000.0,1505500.0,0.0,13218000.0,5261000.0


In [92]:
df_wide[df_wide.iso_code3 == 'IRN'][['Year', 'pop_lvst_initial_pigs']]

,Year,pop_lvst_initial_pigs
996,2011,0.0
997,2012,0.0
998,2013,0.0
999,2014,0.0
1000,2015,0.0
1001,2016,0.0
1002,2017,0.0
1003,2018,0.0
1004,2019,0.0
1005,2020,0.0


In [93]:
# Generate new csv files for each selected livestock item
for sisepuede_var_name in [i for i in df_wide.columns if "pop_lvst" in i]:
    
    if df_wide[sisepuede_var_name].isna().any():   
        df_wide[sisepuede_var_name] = df_wide[sisepuede_var_name].fillna(0)

    # Save the new df in its directory.
    dir_path = f'../AFOLU/{sisepuede_var_name}/input_to_sisepuede'    
    df_wide[["iso_code3","Nation", "Year",sisepuede_var_name]].to_csv(os.path.join(dir_path,f"historical/{sisepuede_var_name}.csv"), index = False)


In [94]:
# Generate projected input data
max_year = df_wide.Year.max()

projected_lvst_df = df_wide[df_wide.Year == max_year]

projected_lvst_df = projected_lvst_df.drop(columns = "Year")

projected_years_df = pd.DataFrame({"Year" : range(max_year+1, 2051)})
projected_lvst_df = projected_lvst_df.merge(projected_years_df, how = "cross") 


for sise_var in [i for i in df_wide.columns if "pop_lvst" in i]:
    dir_path = f'../AFOLU/{sise_var}/input_to_sisepuede'
    projected_lvst_df[["iso_code3","Nation", "Year",sise_var]].to_csv(os.path.join(dir_path,f"projected/{sise_var}.csv"), index = False)